### Install requirements

In [ ]:
# !pip uninstall numpy
!pip install --upgrade numpy==1.23
!pip install pydub
!pip install pyannote.audio

  Using cached numpy-1.23.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.2 kB)
Using cached numpy-1.23.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires numpy==1.22.0; python_version <= "3.10", but you have numpy 1.23.0 which is incompatible.
albucore 0.0.16 requires numpy>=1.24, but you have numpy 1.23.0 which is incompatible.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.23.0 which is incompatible.
bigframes 1.17.0 requires numpy>=1.24.0, but you have numpy 1.23.0 which is incompatible.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.0 which is incompatible.
cudf-cu

ERROR: Operation cancelled by user


In [3]:
!pip install tts

  Using cached numpy-1.22.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.0 kB)
Using cached numpy-1.22.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.16 requires numpy>=1.24, but you have numpy 1.22.0 which is incompatible.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.22.0 which is incompatible.
arviz 0.19.0 requires numpy>=1.23.0, but you have numpy 1.22.0 which is incompatible.
astropy 6.1.3 requires numpy>=1.23, but you have numpy 1.22.0 which is incompatible.
bigframes 1.17.0 requires numpy>=1.24.0, but you have numpy 1.22.0 which is incompatible.
chex 0.1.86 requires numpy>=1.2

In [ ]:
!pip install azure-cognitiveservices-speech azure-ai-textanalytics
!pip install noisereduce soundfile
!pip install speech_recognition
!pip install googletrans
!pip install gtts
!pip install ffmpeg
!pip install demucs

  Using cached azure_cognitiveservices_speech-1.40.0-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached azure_ai_textanalytics-5.3.0-py3-none-any.whl.metadata (82 kB)
  Using cached azure_core-1.31.0-py3-none-any.whl.metadata (39 kB)
  Using cached azure_common-1.1.28-py2.py3-none-any.whl.metadata (5.0 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl.metadata (9.6 kB)
Using cached azure_cognitiveservices_speech-1.40.0-py3-none-manylinux1_x86_64.whl (40.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.6/298.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.4/197.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement speech_recognition (from versions: none)
ERROR: No matching distribution found for speech_recognition
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.4 MB/s eta 0:00

### Import necessary libraries

In [2]:
from pyannote.audio import Pipeline
from pydub import AudioSegment
import os
import os
import azure.cognitiveservices.speech as speechsdk
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
import moviepy.editor as mp
import time
#import noisereduce as nr
import librosa
import soundfile as sf
import numpy as np
from scipy.io.wavfile import write
#import speech_recognition as sr
# from transformers import pipeline
from googletrans import Translator
from gtts import gTTS
from pydub import AudioSegment
import azure.cognitiveservices.speech as speechsdk
import ffmpeg
import subprocess

#### Global variables

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
from TTS.api import TTS

In [4]:
# Azure credentials
speech_key = "***"
speech_region = "eastus"
translator_key = "***"
translator_endpoint = "https://api.cognitive.microsofttranslator.com/"

tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=False)

pipeline = Pipeline.from_pretrained(
        'pyannote/speaker-diarization-3.1',
        use_auth_token='***'
    )

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


## Utils

#### Reduce Noise

In [ ]:
def remove_noise(input_audio_path, output_audio_path, noise_reduction_level=0.1):
    """
    Remove background noise from audio.
    Args:
        input_audio_path (str): Path to the input audio file.
        output_audio_path (str): Path to save the noise-reduced audio file.
        noise_reduction_level (float): The amount of noise reduction (0 to 1).
    """
    # Load the audio file
    audio_data, sample_rate = librosa.load(input_audio_path, sr=None)

    # Perform noise reduction using spectral gating
    reduced_noise = librosa.effects.reduce_noise(y=audio_data, sr=sample_rate, prop_decrease=noise_reduction_level)

    # Save the noise-reduced audio file
    write(output_audio_path, sample_rate, (reduced_noise * 32767).astype(np.int16))
    print(f"Noise-reduced audio saved at {output_audio_path}")

def separate_with_demucs(audio_file_path, output_dir="demucs_output"):
    # Run Demucs to separate vocals and music
    command = ["demucs", "--two-stems=vocals", audio_file_path, "-o", output_dir]
    subprocess.run(command)

#### Video to audio

In [5]:
def extract_audio_from_video(video_file, output_audio_file):
    print("Extracting audio from video...")
    video = mp.VideoFileClip(video_file)
    video.audio.write_audiofile(output_audio_file)
    print(f"Audio saved to {output_audio_file}")

    # Separate audio into vocals and non-vocals
    separate_with_demucs(output_audio_file, "output_audio_folder")

#### Speech to text

In [6]:
def segregate_speakers(audio_path, output_dir="output_segments"):
    """
    Segregates different speaker's voices and returns their timestamps and audio segments.

    :param audio_path: Path to the input audio file.
    :param output_dir: Directory where speaker audio segments will be saved.
    :return: List of dictionaries containing speaker label, timestamps, and segment path.
    """
    # Perform diarization on the input audio file
    diarization = pipeline(audio_path)

    # Load the original audio using pydub
    audio = AudioSegment.from_wav(audio_path)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Create a dictionary to store speaker segments
    speaker_data = {}

    # Iterate over diarization results and save each segment
    for i, (segment,_ , speaker) in enumerate(diarization.itertracks(yield_label=True)):
        # Convert start and end times from seconds to milliseconds
        start_time = segment.start * 1000
        end_time = segment.end * 1000

        # Extract the audio segment for this speaker
        audio_segment = audio[start_time:end_time]

        # Save the audio segment to a file
        segment_filename = os.path.join(output_dir, f"speaker_{speaker}_segment_{i}.wav")
        audio_segment.export(segment_filename, format="wav")

        # Store each segment in the speaker's dictionary
        if speaker not in speaker_data:
            speaker_data[speaker] = []
        speaker_data[speaker].append({
            'start_time': segment.start,
            'end_time': segment.end,
            'audio_path': segment_filename
        })

    return speaker_data

def transcribe_audio_segment(segment_path, source_language):
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.speech_recognition_language = source_language
    audio_config = speechsdk.AudioConfig(filename=segment_path)

    recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    result = recognizer.recognize_once()

    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        return result.text
    else:
        return None

# Function to perform speaker diarization, segment the audio, and transcribe
def transcribe_multispeaker_audio(audio_file_path, source_language, output_dir="output_segments"):
    # Step 1: Segregate speakers and get their audio segments
    speaker_data = segregate_speakers(audio_file_path, output_dir)
    # Step 2: Transcribe each speaker's audio segment
    transcription_results = []
    for speaker, segments in speaker_data.items():
        for segment in segments:
            segment_path = segment['audio_path']
            print(f"Transcribing segment for {speaker}: {segment_path}")
            # Transcribe the segment
            transcription = transcribe_audio_segment(segment_path, source_language)
            # Append the transcription and metadata
            transcription_results.append({
                'speaker': speaker,
                'start_time': segment['start_time'],
                'end_time': segment['end_time'],
                'transcription': transcription,
                'audio_path': segment_path
            })

    # Step 3: Return combined transcription results
    return transcription_results

In [7]:
transcriptions = transcribe_multispeaker_audio("/content/drive/MyDrive/aais-aidub/test.wav", "en-US")

Transcribing segment for SPEAKER_02: output_segments/speaker_SPEAKER_02_segment_0.wav
Transcribing segment for SPEAKER_02: output_segments/speaker_SPEAKER_02_segment_1.wav
Transcribing segment for SPEAKER_02: output_segments/speaker_SPEAKER_02_segment_2.wav
Transcribing segment for SPEAKER_02: output_segments/speaker_SPEAKER_02_segment_3.wav
Transcribing segment for SPEAKER_02: output_segments/speaker_SPEAKER_02_segment_11.wav
Transcribing segment for SPEAKER_02: output_segments/speaker_SPEAKER_02_segment_12.wav
Transcribing segment for SPEAKER_01: output_segments/speaker_SPEAKER_01_segment_4.wav
Transcribing segment for SPEAKER_01: output_segments/speaker_SPEAKER_01_segment_5.wav
Transcribing segment for SPEAKER_01: output_segments/speaker_SPEAKER_01_segment_6.wav
Transcribing segment for SPEAKER_01: output_segments/speaker_SPEAKER_01_segment_8.wav
Transcribing segment for SPEAKER_00: output_segments/speaker_SPEAKER_00_segment_7.wav
Transcribing segment for SPEAKER_00: output_segments

#### Translate

In [18]:
def translate(transcription, target_language):
    import requests
    import uuid

    # Prepare the translation request
    path = '/translate?api-version=3.0'
    params = f'&to={target_language}'
    constructed_url = translator_endpoint + path + params

    headers = {
        'Ocp-Apim-Subscription-Key': translator_key,
        'Ocp-Apim-Subscription-Region': speech_region,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    body = [{'text': transcription}]

    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()

    translated_text = response[0]["translations"][0]["text"]
    return translated_text

def translate_text(transcriptions, target_language):
    for transcription in transcriptions:
        if transcription['transcription']:
            translated_text = translate(transcription['transcription'], target_language)
            transcription['translated_text'] = translated_text
        else:
            transcription['translated_text'] = transcription['transcription']
    return transcriptions

In [19]:
transcriptions = translate_text(transcriptions, "hi")

In [21]:
transcriptions

[{'speaker': 'SPEAKER_02',
  'start_time': 0.03096875,
  'end_time': 8.130968750000001,
  'transcription': 'Well, the vexed issue of euthanasia continues to divide the community. Few debates have been as emotionally charged, and never have two sides been further apart.',
  'audio_path': 'output_segments/speaker_SPEAKER_02_segment_0.wav',
  'translated_text': 'खैर, इच्छामृत्यु का विवादास्पद मुद्दा समुदाय को विभाजित करना जारी रखता है। कुछ बहसें भावनात्मक रूप से चार्ज की गई हैं, और कभी भी दो पक्ष अलग नहीं हुए हैं।'},
 {'speaker': 'SPEAKER_02',
  'start_time': 8.704718750000001,
  'end_time': 19.656593750000003,
  'transcription': 'Tonight we look at both those sites, this controversial issue with Ronald Marks, head of the Euthanasia Council of Australia. Good evening and Vanessa Gibbs, acting chairperson of the Civil Liberties Council.',
  'audio_path': 'output_segments/speaker_SPEAKER_02_segment_1.wav',
  'translated_text': 'आज रात हम उन दोनों साइटों को देखते हैं, ऑस्ट्रेलिया के इच्छामृत

#### Text to Speech

In [27]:
def text_to_speech(transcriptions, target_language):
    for transcription in transcriptions:
        translated_text = transcription['translated_text']
        original_audio = transcription['audio_path']
        output_file = transcription['audio_path'].replace(".wav", "_dubbed.wav")
        print(translated_text)
        if translated_text:
            try:
                tts.tts_to_file(text=translated_text, file_path=output_file, speaker_wav=original_audio, language=target_language)
                transcription['dubbed_audio_path'] = output_file
            except:
                transcription['dubbed_audio_path'] = original_audio
        else:
            transcription['dubbed_audio_path'] = original_audio

    return transcriptions

In [41]:
transcriptions

[{'speaker': 'SPEAKER_02',
  'start_time': 0.03096875,
  'end_time': 8.130968750000001,
  'transcription': 'Well, the vexed issue of euthanasia continues to divide the community. Few debates have been as emotionally charged, and never have two sides been further apart.',
  'audio_path': 'output_segments/speaker_SPEAKER_02_segment_0.wav',
  'translated_text': 'खैर, इच्छामृत्यु का विवादास्पद मुद्दा समुदाय को विभाजित करना जारी रखता है। कुछ बहसें भावनात्मक रूप से चार्ज की गई हैं, और कभी भी दो पक्ष अलग नहीं हुए हैं।',
  'dubbed_audio_path': 'output_segments/speaker_SPEAKER_02_segment_0_dubbed.wav'},
 {'speaker': 'SPEAKER_02',
  'start_time': 8.704718750000001,
  'end_time': 19.656593750000003,
  'transcription': 'Tonight we look at both those sites, this controversial issue with Ronald Marks, head of the Euthanasia Council of Australia. Good evening and Vanessa Gibbs, acting chairperson of the Civil Liberties Council.',
  'audio_path': 'output_segments/speaker_SPEAKER_02_segment_1.wav',
  '

In [42]:
sorted_data = sorted(transcriptions, key=lambda x: x['start_time'])

In [ ]:
text_to_speech(transcriptions, "hi")

#### Replace original audio with dubbed audio in Video


In [ ]:
def mute_original_audio(input_video_path, output_muted_video_path):
    """
    Mute the original audio in the video.

    Args:
        input_video_path (str): Path to the input video file.
        output_muted_video_path (str): Path to save the muted output video.
    """
    subprocess.run([
        'ffmpeg', '-i', input_video_path, '-c', 'copy', '-an', output_muted_video_path
    ], check=True)
    print(f"Muted video saved at: {output_muted_video_path}")

def replace_audio(input_video_path, input_audio_path, output_video_path):
    """
    Replace the original audio in a video with new audio.

    Args:
        input_video_path (str): Path to the input muted video file.
        input_audio_path (str): Path to the new audio file.
        output_video_path (str): Path to save the final video with replaced audio.
    """
    subprocess.run([
        'ffmpeg', '-i', input_video_path, '-i', input_audio_path, '-c:v', 'copy', '-c:a', 'aac', '-strict', 'experimental', output_video_path
    ], check=True)
    print(f"Final video with replaced audio saved at: {output_video_path}")

In [44]:
from pydub import AudioSegment

def merge_audio_with_dubbed_segments(original_audio_path, segments):
    # Load the original audio
    combined_audio = AudioSegment.from_wav(original_audio_path)

    # Process each segment
    for segment in segments:
        start_time_ms = segment['start_time'] * 1000  # Convert to milliseconds
        end_time_ms = segment['end_time'] * 1000      # Convert to milliseconds
        dubbed_audio_path = segment['dubbed_audio_path']
        # Load the dubbed audio
        dubbed_audio = AudioSegment.from_wav(dubbed_audio_path)
        # Overlay the dubbed audio onto the original audio at the start time
        combined_audio = combined_audio.overlay(dubbed_audio, position=start_time_ms)

    # Export the combined audio
    output_path = "merged_output.wav"
    combined_audio.export(output_path, format="wav")

    return output_path

merge_audio_with_dubbed_segments("/content/demucs_output/htdemucs/singam_audio/no_vocals.wav", transcriptions)

'merged_output.wav'

#### Change audio Speed

In [ ]:
from pydub import AudioSegment
from pydub.playback import speedup

def adjust_audio_speed(audio_path, start_time, end_time, output_path):
    # Load the audio file
    audio = AudioSegment.from_file(audio_path)

    # Calculate the original duration of the audio in seconds
    original_duration = len(audio) / 1000.0  # Convert from milliseconds to seconds

    # Calculate the target duration (difference between start and end time)
    target_duration = end_time - start_time

    # Calculate the speed factor (target duration / original duration)
    speed_factor = original_duration / target_duration

    # Speed up or slow down the audio to fit the target duration
    if speed_factor > 1:
        # If the speed factor is greater than 1, speed up the audio
        adjusted_audio = speedup(audio, playback_speed=speed_factor)

    # Export the adjusted audio
    adjusted_audio.export(audio_path, format="wav")

    print(f"Audio adjusted to fit between {start_time}s and {end_time}s and saved as {output_path}")
    print(f"Original Duration: {original_duration}s, Target Duration: {target_duration}s")

In [31]:
!cp -r /content/output_segments /content/drive/MyDrive/aais-aidub

# Dub (main function)

In [ ]:
input_video_file = "/content/drive/MyDrive/aais-aidub/news_video.mp4"
original_audio = "/content/drive/MyDrive/aais-aidub/news_video_ori_audio.wav"
output_audio = "/content/drive/MyDrive/aais-aidub/news_video_output_audio.wav"
output_video = "/content/drive/MyDrive/aais-aidub/news_video_dubbed.mp4"
target_language = "hi"
source_language = "en-US"

# Step 1: Extract the original audio from the video
original_audio_file = input_video_file.replace(".mp4", "_ori_audio.wav")
extract_audio_from_video(input_video_file, original_audio_file)

# Step 2: Transcribe the original audio in the specified source language
transcription = transcribe_audio(original_audio_file, source_language)
if not transcription:
    print("Transcription failed.")

In [ ]:
# Main function to run the process
def DubVideo(input_video_file, target_language, source_language="en-US"):

    # Step 1: Extract the original audio from the video
    original_audio_file = input_video_file.split(".mp4")[0] + ".wav"
    extract_audio_from_video(input_video_file, original_audio_file)

    # Step 2: Transcribe the original audio in the specified source language
    transcription = transcribe_audio(original_audio_file, source_language)
    if not transcription:
        print("Transcription failed.")
        return

    # Step 3: Translate the transcription into the target language
    translated_text = translate_text(transcription, target_language)

    # Step 4: Convert the translated text back to speech
    output_audio_file = original_audio_file.split(".wav")[0] + "_dubbed.wav"
    text_to_speech(translated_text, output_audio_file, original_audio_file, target_language)

    # Step 2: Reduce noise in the extracted audio
    reduce_noise(output_audio_file, output_audio_file)

    # Step 5: Replace the audio in the original video
    output_video_file = input_video_file.split(".mp4")[0] + "_dubbed.wav"
    replace_audio_in_video(input_video_file, output_audio_file, output_video_file)